## Exponential Moving Average

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [2]:
AAPL = pd.read_csv('AAPL_New.csv')
AAPL.set_index('Date')
AAPL['OpenTmr'] = AAPL['Open'].shift(-1)
AAPL['OpenClose'] = (AAPL['Open']+ AAPL['Close'])/2
AAPL['HighLow'] = (AAPL['High']+ AAPL['Low'])/2
AAPL['OCHL'] = (AAPL['Open']+ AAPL['Close']+AAPL['High']+ AAPL['Low'])/4
# AAPL = AAPL.fillna(method='ffill')
AAPL.tail(10)
# AAPL = AAPL.dropna(0)

,Date,High,Low,Open,Close,Volume,Adj Close,OpenTmr,OpenClose,HighLow,OCHL
978,11/20/2019,266.079987,260.399994,265.540008,263.190002,26558600,263.190002,263.690002,264.365005,263.239990,263.802498
979,11/21/2019,264.010010,261.179993,263.690002,262.010010,30348800,262.010010,262.589996,262.850006,262.595001,262.722504
980,11/22/2019,263.179993,260.839996,262.589996,261.779999,16331300,261.779999,262.709992,262.184998,262.009994,262.097496
981,11/25/2019,266.440002,262.519989,262.709992,266.369995,21005100,266.369995,266.940002,264.539993,264.479996,264.509995
982,11/26/2019,267.160004,262.500000,266.940002,264.290008,26301900,264.290008,265.579987,265.615005,264.830002,265.222504
983,11/27/2019,267.980011,265.309998,265.579987,267.839996,16308900,267.839996,266.600006,266.709991,266.645004,266.677498
984,11/29/2019,268.000000,265.899994,266.600006,267.250000,11654400,267.250000,267.269989,266.925003,266.949997,266.937500
985,12/2/2019,268.250000,263.450012,267.269989,264.160004,23621800,264.160004,258.309998,265.714996,265.850006,265.782501
986,12/3/2019,259.529999,256.290008,258.309998,259.450012,28607600,259.450012,261.070007,258.880005,257.910004,258.395004
987,12/4/2019,263.309998,260.679993,261.070007,261.739990,15874176,261.739990,NaN,261.404999,261.994995,261.699997


In [3]:
N = AAPL.shape[0] # total num days
num_boot = 300# total num bootstrap
T= 250 # start day
window = 200 # training period window

In [4]:
df = AAPL[['Close']]
df.reset_index(level=0, inplace=True)
df.columns=['ds','y']

# exponential weighted functions
X = df.y.ewm(span=5, adjust=False).mean()

Y =AAPL['OpenTmr'][-(N-(T+1)):].values
#X = np.column_stack([np.ones((T,1)),X])
print(X.shape)
print(Y.shape)
#movAverage1= (movAvg) 

(988,)
(737,)


In [5]:
def EMA(X, Y):
    
    T = X.shape[0]
    #print(T)
    #mu = (AAPL['Open'].mean(),AAPL['Open '].mean(),AAPL['Open '].mean()) 
    #cov = [[1,0.75,-0.35],[0.75,1,0.9],[-0.35,0.9,1]]
    #F = np.random.multivariate_normal(mu,cov,T)
    #Sample for Y,X
    X = np.column_stack([np.ones((T,1)),X])
    #T = X.shape[0]
    N = X.shape
    #beta = np.array([0.56,2.53,2.05,1.78])
    #beta.shape=(N[1],1)
    #Y =X@beta+np.random.normal(0,1,(T,1))
    #Y=AAPL['Open '].values
    #print(T)
    invXX = np.linalg.inv(X.transpose()@X)
    beta_hat = invXX@X.transpose()@Y
    y_hat = X@beta_hat
    residuals = Y-y_hat
    sigma2 = (1/T)*residuals.transpose()@residuals

    sigma = np.sqrt(sigma2)

    #variance - covariance of beta_hat
    varcov_beta_hat = (sigma2)*invXX
    std_beta_hat = np.sqrt(T*np.diag(varcov_beta_hat))

    R_square = 1-(residuals.transpose()@residuals)/(T*np.var(Y))

    adj_R_square = 1-(1-R_square)*(T-1)/(T - N[1])

    #Testing Coefficents:beta_i
    #Null Hypotesis

    t_stat = (beta_hat.transpose()-0)/std_beta_hat
    p_val = 1-ss.norm.cdf(t_stat)

    #Test of joint significance
    F_stat= (beta_hat.transpose()@np.linalg.inv(varcov_beta_hat)@beta_hat/N[1])/(residuals.transpose()@residuals/(T-N[1]))
    p_val_F= 1 - ss.f.cdf(F_stat,N[1]-1, T-N[1])
    
    return beta_hat,y_hat


In [6]:
def ema_bootstrap():
    T = 250
    #print(T)
    N = X.shape[0]
    #print(N)
    
    yhat_ema = np.zeros(N-(T+1))
    window = 200
    num_boost = 300 # increase
    
    for t in range(T+1,N):
        X_train = df.y.ewm(span=5, adjust=False).mean()[t-window:t-1]
        #X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
        Y_train = AAPL[['OpenTmr']][t-window:t-1].values
        #print(X_train.shape)
        #print(Y_train.shape)
        X_pred = df.y.ewm(span=5, adjust=False).mean()[t-1:t]
        X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
        yhat_train = EMA(X_train , Y_train)[1]
        res_train = Y_train - yhat_train
       
        y_pred_all = np.zeros(num_boost)
       
        for i in range (0,num_boost):
            #err = np.random.choice(res_train,(window-1, ),replace = True)
            err = res_train
            y_bstr = yhat_train + err
            beta_bstr = EMA(X_train,y_bstr)[0]
            #print(X_pred.shape)
            #print(beta_bstr.shape)
 
            y_pred_bstr = X_pred@beta_bstr
            y_pred_all[i] = y_pred_bstr
           
        y_pred_ema = y_pred_all.mean()
       
        yhat_ema[t-(T+1)] = y_pred_ema
   
   
    rmse_ema = np.sqrt(np.mean((Y[:-1] - yhat_ema[:-1])**2))
   
    return yhat_ema,rmse_ema

In [7]:
y, rmse = ema_bootstrap()
rmse

4.2342257138635535

In [8]:
len(y)

737

In [9]:
AAPL.Date.iloc[-200:]

788     2/21/2019
789     2/22/2019
790     2/25/2019
791     2/26/2019
792     2/27/2019
          ...    
983    11/27/2019
984    11/29/2019
985     12/2/2019
986     12/3/2019
987     12/4/2019
Name: Date, Length: 200, dtype: object

In [10]:
y[-200:]

array([171.00163176, 170.92302894, 171.51557676, 172.34494423,
       172.9226825 , 173.47835332, 173.27945039, 173.77046284,
       174.40491719, 174.71520116, 174.57844109, 173.79977787,
       173.39100764, 175.16607892, 177.03506678, 178.56107436,
       180.26691468, 182.20619992, 184.13697605, 184.93122527,
       185.98137515, 189.00290081, 189.68495309, 189.35499894,
       188.50374659, 188.49013675, 188.56554719, 189.02727015,
       189.77575002, 191.20622989, 192.61826328, 193.69247031,
       194.84734276, 196.64611592, 197.66225338, 198.7169791 ,
       198.87121874, 198.93349694, 199.09570799, 199.20628922,
       200.58473319, 201.74840318, 202.75047854, 204.40927673,
       205.40727723, 205.43762428, 205.13020278, 205.02096048,
       203.61780556, 206.01990353, 207.17169842, 208.81206858,
       208.76854108, 206.85627976, 205.58081502, 203.98525239,
       201.72145499, 196.33748597, 193.72853631, 192.74707824,
       191.82836762, 190.79645611, 188.11658606, 187.48

In [11]:
EMA_predict = pd.DataFrame()
EMA_predict['Date'] = AAPL.Date.iloc[-200:]
EMA_predict['Predict_EMA'] = y[-200:]

In [12]:
EMA_predict

,Date,Predict_EMA
788,2/21/2019,171.001632
789,2/22/2019,170.923029
790,2/25/2019,171.515577
791,2/26/2019,172.344944
792,2/27/2019,172.922683
...,...,...
983,11/27/2019,266.685224
984,11/29/2019,267.879824
985,12/2/2019,268.470260
986,12/3/2019,267.781034


In [13]:
EMA_predict.to_csv('EMA_prediction.csv')